# Populador de Banco de Dados - Bluma Case

## 📋 Visão Geral

Este notebook implementa a população do banco de dados `bluma_case` com dados sintéticos realistas para um marketplace de beleza e bem-estar. 

### Objetivos:
- Gerar 50.000 usuários com características demográficas brasileiras
- Criar ~150.000 pedidos com padrões realistas de compra
- Gerar 120 campanhas de mídia paga (Meta, Google, TikTok)
- Simular performance diária das campanhas com sazonalidade
- Criar dados de criativos e eventos de usuários
- Implementar análise de cohorts e alocação de budget

### Estrutura do Banco:
- **users**: Dados demográficos e de aquisição
- **orders**: Pedidos e transações
- **paid_media_campaigns**: Campanhas de mídia paga
- **daily_performance**: Performance diária das campanhas
- **ad_creatives**: Criativos das campanhas
- **creative_performance**: Performance dos criativos
- **user_events**: Eventos de interação dos usuários
- **user_cohorts**: Análise de cohorts
- **budget_allocation**: Alocação mensal de budget

## 1. Database Connection Setup

Importação das bibliotecas necessárias e configuração da conexão com o banco de dados MySQL.

In [2]:
# Instalar dependências necessárias
# Execute esta célula primeiro para instalar os pacotes necessários

import subprocess
import sys

def install_package(package):
    """Instala um pacote usando pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} instalado com sucesso!")
    except subprocess.CalledProcessError as e:
        print(f"❌ Erro ao instalar {package}: {e}")

# Lista de pacotes necessários
required_packages = [
    "pandas",
    "numpy", 
    "faker",
    "mysql-connector-python"
]

print("🔧 Instalando dependências necessárias...")
print("=" * 50)

for package in required_packages:
    install_package(package)

print("\n✅ Instalação concluída! Agora você pode executar as demais células.")

🔧 Instalando dependências necessárias...
✅ pandas instalado com sucesso!
✅ numpy instalado com sucesso!
✅ faker instalado com sucesso!
✅ mysql-connector-python instalado com sucesso!

✅ Instalação concluída! Agora você pode executar as demais células.


In [3]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from faker import Faker
import mysql.connector
from datetime import datetime, timedelta
import random
import uuid
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# Configurar faker para português brasileiro
fake = Faker('pt_BR')
Faker.seed(42)  # Para reproducibilidade
np.random.seed(42)
random.seed(42)

# Configuração da conexão com o banco
DB_CONFIG = {
    'host': '95.111.240.159',
    'database': 'bluma_case',
    'user': 'root',
    'password': 'rafa906996'
}

print("✅ Bibliotecas importadas com sucesso!")
print(f"📊 Faker configurado para: {fake.locale}")
print(f"🗄️ Banco configurado: {DB_CONFIG['database']} @ {DB_CONFIG['host']}")

✅ Bibliotecas importadas com sucesso!
📊 Faker configurado para: <bound method BaseProvider.locale of <faker.providers.user_agent.Provider object at 0x000002066A25CC20>>
🗄️ Banco configurado: bluma_case @ 95.111.240.159


In [4]:
def create_connection():
    """
    Estabelece conexão com o banco de dados MySQL.
    
    Returns:
        mysql.connector.connection: Objeto de conexão com o banco
    """
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        if connection.is_connected():
            print("✅ Conexão com o banco estabelecida com sucesso!")
            print(f"🔗 Versão do servidor MySQL: {connection.get_server_info()}")
            return connection
    except mysql.connector.Error as error:
        print(f"❌ Erro ao conectar com o banco: {error}")
        return None

# Testar conexão
test_connection = create_connection()
if test_connection:
    test_connection.close()
    print("🔄 Conexão testada e fechada com sucesso!")

✅ Conexão com o banco estabelecida com sucesso!
🔗 Versão do servidor MySQL: 8.0.42
🔄 Conexão testada e fechada com sucesso!


## 2. Helper Functions for Data Generation

Funções utilitárias para inserção em lote, limpeza de tabelas e helpers estatísticos para gerar dados realistas.

In [19]:
def truncate_tables(connection):
    """
    Limpa todas as tabelas antes de popular com novos dados.
    
    Args:
        connection: Conexão com o banco de dados
    """
    cursor = connection.cursor()
    
    # Lista das tabelas na ordem de dependência (FK constraints)
    tables = [
        'creative_performance',
        'ad_creatives', 
        'daily_performance',
        'user_events',
        'user_cohorts',
        'budget_allocation',
        'orders',
        'paid_media_campaigns',
        'users'
    ]
    
    try:
        # Desabilitar foreign keys temporariamente
        cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
        
        for table in tables:
            cursor.execute(f"TRUNCATE TABLE {table};")
            print(f"🗑️ Tabela {table} limpa")
        
        # Re-habilitar foreign keys 
        cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
        connection.commit()
        print("✅ Todas as tabelas foram limpas com sucesso!")
        
    except mysql.connector.Error as error:
        print(f"❌ Erro ao limpar tabelas: {error}")
    finally:
        cursor.close()

def batch_insert(connection, table_name: str, data: List[Dict], batch_size: int = 1000):
    """
    Insere dados em lotes para melhor performance.
    
    Args:
        connection: Conexão com o banco de dados
        table_name: Nome da tabela
        data: Lista de dicionários com os dados
        batch_size: Tamanho do lote (default: 1000)
    """
    if not data:
        print(f"⚠️ Nenhum dado para inserir na tabela {table_name}")
        return
        
    cursor = connection.cursor()
    
    # Pegar as colunas do primeiro item
    columns = list(data[0].keys())
    placeholders = ', '.join(['%s'] * len(columns))
    
    insert_query = f"""
    INSERT INTO {table_name} ({', '.join(columns)}) 
    VALUES ({placeholders})
    """
    
    total_records = len(data)
    processed = 0
    
    try:
        for i in range(0, total_records, batch_size):
            batch = data[i:i + batch_size]
            
            # Converter dados para tuplas
            batch_values = []
            for record in batch:
                values = []
                for col in columns:
                    value = record[col]
                    # Tratar valores None
                    if value is None:
                        values.append(None)
                    # Tratar datetime
                    elif isinstance(value, datetime):
                        values.append(value.strftime('%Y-%m-%d %H:%M:%S'))
                    # Tratar numpy types
                    elif hasattr(value, 'item'):  # numpy scalar
                        values.append(value.item())
                    # Converter para string se necessário (UUID, etc.)
                    else:
                        values.append(str(value) if not isinstance(value, (int, float, bool)) else value)
                batch_values.append(tuple(values))
            
            cursor.executemany(insert_query, batch_values)
            processed += len(batch)
            
            print(f"📊 {table_name}: {processed}/{total_records} registros inseridos ({(processed/total_records)*100:.1f}%)")
        
        connection.commit()
        print(f"✅ {table_name}: {total_records} registros inseridos com sucesso!")
        
    except mysql.connector.Error as error:
        print(f"❌ Erro ao inserir dados na tabela {table_name}: {error}")
        connection.rollback()
    finally:
        cursor.close()

print("✅ Funções auxiliares atualizadas!")

✅ Funções auxiliares atualizadas!


In [6]:
# Constantes de negócio conforme especificação
BUSINESS_PARAMS = {
    'acquisition_channels': {
        'Meta Ads': {'weight': 0.45, 'cac': 85, 'activation_rate': 0.078},
        'Google Ads': {'weight': 0.25, 'cac': 95, 'activation_rate': 0.092},
        'TikTok Ads': {'weight': 0.10, 'cac': 75, 'activation_rate': 0.065},
        'Organic': {'weight': 0.15, 'cac': 0, 'activation_rate': 0.15},
        'Referral': {'weight': 0.05, 'cac': 30, 'activation_rate': 0.25}
    },
    'services': {
        'Manicure': {'weight': 0.35, 'avg_ticket': 65, 'frequency_per_month': 2.5},
        'Massagem': {'weight': 0.20, 'avg_ticket': 120, 'frequency_per_month': 1.2},
        'Limpeza de Pele': {'weight': 0.15, 'avg_ticket': 95, 'frequency_per_month': 1.5},
        'Design Sobrancelhas': {'weight': 0.15, 'avg_ticket': 55, 'frequency_per_month': 2.0},
        'Depilação': {'weight': 0.15, 'avg_ticket': 80, 'frequency_per_month': 1.8}
    },
    'cities': {
        'São Paulo': {'weight': 0.30, 'state': 'SP'},
        'Rio de Janeiro': {'weight': 0.20, 'state': 'RJ'},
        'Belo Horizonte': {'weight': 0.10, 'state': 'MG'},
        'Brasília': {'weight': 0.08, 'state': 'DF'},
        'Curitiba': {'weight': 0.07, 'state': 'PR'},
        'Porto Alegre': {'weight': 0.05, 'state': 'RS'},
        'Salvador': {'weight': 0.05, 'state': 'BA'},
        'Fortaleza': {'weight': 0.05, 'state': 'CE'},
        'Recife': {'weight': 0.05, 'state': 'PE'},
        'Campinas': {'weight': 0.05, 'state': 'SP'}
    },
    'creative_types': {
        'UGC': {'ctr': 0.028, 'cvr': 0.012},
        'Carousel': {'ctr': 0.019, 'cvr': 0.010},
        'Video': {'ctr': 0.022, 'cvr': 0.011},
        'Static': {'ctr': 0.015, 'cvr': 0.008},
        'ASMR': {'ctr': 0.031, 'cvr': 0.009}
    }
}

# Períodos de tempo
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2025, 1, 31)

def get_seasonal_factor(date: datetime) -> float:
    """
    Retorna fator de ajuste sazonal para uma data específica.
    
    Args:
        date: Data para calcular o fator
        
    Returns:
        float: Fator multiplicativo (1.0 = normal)
    """
    month = date.month
    day = date.day
    
    # Dia das mães (maio): +50%
    if month == 5:
        return 1.5
    
    # Black Friday (novembro 20+): +80%
    if month == 11 and day >= 20:
        return 1.8
    
    # Dezembro (até 25): +40%
    if month == 12 and day <= 25:
        return 1.4
    
    return 1.0

def get_weekday_factor(date: datetime) -> float:
    """
    Retorna fator de ajuste por dia da semana.
    
    Args:
        date: Data para calcular o fator
        
    Returns:
        float: Fator multiplicativo
    """
    weekday = date.weekday()  # 0 = segunda, 6 = domingo
    
    if weekday == 4:  # Sexta-feira: +15%
        return 1.15
    elif weekday in [5, 6]:  # Fins de semana: -20%
        return 0.8
    else:
        return 1.0

def add_gaussian_noise(value: float, noise_factor: float) -> float:
    """
    Adiciona ruído gaussiano a um valor.
    
    Args:
        value: Valor base
        noise_factor: Fator de ruído (ex: 0.1 para ±10%)
        
    Returns:
        float: Valor com ruído
    """
    noise = np.random.normal(0, noise_factor)
    return max(0, value * (1 + noise))

print("✅ Constantes de negócio e helpers estatísticos definidos!")
print(f"📅 Período de dados: {START_DATE.strftime('%d/%m/%Y')} até {END_DATE.strftime('%d/%m/%Y')}")

✅ Constantes de negócio e helpers estatísticos definidos!
📅 Período de dados: 01/01/2024 até 31/01/2025


## 3. User Data Generation

Geração de 50.000 usuários com demografias brasileiras realistas, distribuição temporal usando beta distribution e canais de aquisição com taxas de ativação específicas.

In [7]:
def generate_users(num_users: int = 50000) -> List[Dict]:
    """
    Gera dados de usuários com características demográficas brasileiras.
    
    Args:
        num_users: Número de usuários a gerar (default: 50000)
        
    Returns:
        List[Dict]: Lista de usuários gerados
    """
    print(f"🚀 Gerando {num_users:,} usuários...")
    
    users = []
    channels = list(BUSINESS_PARAMS['acquisition_channels'].keys())
    channel_weights = [BUSINESS_PARAMS['acquisition_channels'][ch]['weight'] for ch in channels]
    
    cities = list(BUSINESS_PARAMS['cities'].keys())
    city_weights = [BUSINESS_PARAMS['cities'][city]['weight'] for city in cities]
    
    # Distribuição temporal usando beta distribution (mais usuários nos meses recentes)
    time_range = (END_DATE - START_DATE).days
    beta_samples = np.random.beta(2, 5, num_users)
    
    for i in range(num_users):
        # Data de criação (beta distribution para simular crescimento)
        days_offset = int(beta_samples[i] * time_range)
        created_at = START_DATE + timedelta(days=days_offset)
        
        # Canal de aquisição
        channel = np.random.choice(channels, p=channel_weights)
        
        # Cidade e estado
        city = np.random.choice(cities, p=city_weights)
        state = BUSINESS_PARAMS['cities'][city]['state']
        
        # Demografia: 85% feminino, faixa 25-34 anos predominante
        gender = 'F' if random.random() < 0.85 else 'M'
        
        age_groups = ['18-24', '25-34', '35-44', '45-54', '55+']
        age_weights = [0.15, 0.50, 0.25, 0.08, 0.02]  # 25-34 predominante
        age_group = np.random.choice(age_groups, p=age_weights)
        
        # Calcular first_order_date baseado na taxa de ativação
        activation_rate = BUSINESS_PARAMS['acquisition_channels'][channel]['activation_rate']
        
        first_order_date = None
        if random.random() < activation_rate:
            # Ajuste sazonal: +30% ativação em maio, novembro e dezembro
            seasonal_boost = 1.0
            if created_at.month in [5, 11, 12]:
                seasonal_boost = 1.3
                
            # Se deve ativar, calcular data usando gamma distribution (média 4 dias)
            if random.random() < seasonal_boost:
                days_to_order = max(0, int(np.random.gamma(2, 2)))  # Média ~4 dias
                first_order_date = created_at + timedelta(days=days_to_order)
                
                # Não pode ser no futuro
                if first_order_date > END_DATE:
                    first_order_date = None
        
        # Campanha de aquisição (simplificada)
        if channel in ['Meta Ads', 'Google Ads', 'TikTok Ads']:
            acquisition_campaign = f"{channel.replace(' ', '_').lower()}_prospecting_{random.randint(1, 10)}"
        else:
            acquisition_campaign = channel.lower()
        
        user = {
            'user_id': str(uuid.uuid4()),
            'created_at': created_at,
            'acquisition_channel': channel,
            'acquisition_campaign': acquisition_campaign,
            'first_order_date': first_order_date,
            'city': city,
            'state': state,
            'age_group': age_group,
            'gender': gender
        }
        
        users.append(user)
        
        if (i + 1) % 10000 == 0:
            print(f"👥 {i + 1:,} usuários gerados...")
    
    # Estatísticas
    activated_users = sum(1 for u in users if u['first_order_date'] is not None)
    activation_rate = (activated_users / num_users) * 100
    
    print(f"✅ {num_users:,} usuários gerados!")
    print(f"📊 Usuários ativados: {activated_users:,} ({activation_rate:.1f}%)")
    
    return users

# Gerar usuários
users_data = generate_users()

🚀 Gerando 50,000 usuários...
👥 10,000 usuários gerados...
👥 20,000 usuários gerados...
👥 30,000 usuários gerados...
👥 40,000 usuários gerados...
👥 50,000 usuários gerados...
✅ 50,000 usuários gerados!
📊 Usuários ativados: 4,955 (9.9%)


## 4. Order Data Generation

Criação de pedidos para usuários ativados com padrões realistas de distribuição de serviços, métodos de pagamento, descontos e taxas de cancelamento.

In [8]:
def generate_orders(users: List[Dict]) -> List[Dict]:
    """
    Gera pedidos para usuários ativados.
    
    Args:
        users: Lista de usuários gerados
        
    Returns:
        List[Dict]: Lista de pedidos gerados
    """
    print("🛒 Gerando pedidos...")
    
    # Filtrar apenas usuários ativados
    activated_users = [u for u in users if u['first_order_date'] is not None]
    print(f"👥 {len(activated_users):,} usuários ativados encontrados")
    
    orders = []
    services = list(BUSINESS_PARAMS['services'].keys())
    service_weights = [BUSINESS_PARAMS['services'][svc]['weight'] for svc in services]
    
    # Métodos de pagamento com evolução temporal
    payment_methods = ['PIX', 'Cartão de Crédito', 'Cartão de Débito', 'Dinheiro']
    
    for user in activated_users:
        user_id = user['user_id']
        first_order_date = user['first_order_date']
        
        # Distribuição de pedidos por usuário
        num_orders_rand = random.random()
        if num_orders_rand < 0.30:  # 30%: apenas 1 pedido
            num_orders = 1
        elif num_orders_rand < 0.60:  # 30%: 2-3 pedidos
            num_orders = random.randint(2, 3)
        elif num_orders_rand < 0.85:  # 25%: 4-7 pedidos
            num_orders = random.randint(4, 7)
        else:  # 15%: 8+ pedidos (power users)
            num_orders = random.randint(8, 15)
        
        previous_service = None
        order_date = first_order_date
        
        for order_num in range(num_orders):
            # Tendência a repetir o mesmo serviço (60% de chance)
            if previous_service and random.random() < 0.60:
                service_type = previous_service
            else:
                service_type = np.random.choice(services, p=service_weights)
            
            previous_service = service_type
            
            # Valor do pedido baseado no ticket médio do serviço
            base_value = BUSINESS_PARAMS['services'][service_type]['avg_ticket']
            order_value = add_gaussian_noise(base_value, 0.15)  # ±15% variação
            
            # Desconto no primeiro pedido (40% chance, 15-25% desconto)
            discount_amount = 0.0
            if order_num == 0 and random.random() < 0.40:
                discount_percent = random.uniform(0.15, 0.25)
                discount_amount = order_value * discount_percent
                order_value -= discount_amount
            
            # Status: Taxa de cancelamento (12% novos usuários, 8% recorrentes)
            cancel_rate = 0.12 if order_num == 0 else 0.08
            status = 'cancelled' if random.random() < cancel_rate else 'completed'
            
            # Método de pagamento com evolução temporal
            year_progress = (order_date.year - 2024) + (order_date.month - 1) / 12
            pix_growth = 0.25 + (year_progress * 0.10)  # 25% em 2024 → 35% em 2025
            pix_growth = min(pix_growth, 0.35)
            
            payment_rand = random.random()
            if payment_rand < pix_growth:
                payment_method = 'PIX'
            elif payment_rand < pix_growth + 0.45:
                payment_method = 'Cartão de Crédito'
            elif payment_rand < pix_growth + 0.70:
                payment_method = 'Cartão de Débito'
            else:
                payment_method = 'Dinheiro'
            
            order = {
                'order_id': str(uuid.uuid4()),
                'user_id': user_id,
                'order_date': order_date,
                'order_value': round(order_value, 2),
                'service_type': service_type,
                'status': status,
                'payment_method': payment_method,
                'discount_amount': round(discount_amount, 2)
            }
            
            orders.append(order)
            
            # Calcular próxima data do pedido baseada na frequência do serviço
            if order_num < num_orders - 1:
                frequency_per_month = BUSINESS_PARAMS['services'][service_type]['frequency_per_month']
                days_between_orders = int(30 / frequency_per_month)
                days_variation = random.randint(-5, 10)  # Variação aleatória
                
                next_order_date = order_date + timedelta(days=days_between_orders + days_variation)
                
                # Não ultrapassar o período de dados
                if next_order_date <= END_DATE:
                    order_date = next_order_date
                else:
                    break
    
    # Estatísticas
    completed_orders = [o for o in orders if o['status'] == 'completed']
    total_gmv = sum(o['order_value'] for o in completed_orders)
    
    print(f"✅ {len(orders):,} pedidos gerados!")
    print(f"📊 Pedidos completados: {len(completed_orders):,}")
    print(f"💰 GMV Total: R$ {total_gmv:,.2f}")
    
    return orders

# Gerar pedidos
orders_data = generate_orders(users_data)

🛒 Gerando pedidos...
👥 4,955 usuários ativados encontrados
✅ 19,683 pedidos gerados!
📊 Pedidos completados: 17,919
💰 GMV Total: R$ 1,430,718.42


## 5. Campaign Data Generation

Geração de 120 campanhas de mídia paga distribuídas entre Meta (50%), Google (35%) e TikTok (15%) com diferentes tipos de campanha, budgets e durações realistas.

In [10]:
def generate_campaigns(num_campaigns: int = 120) -> List[Dict]:
    """
    Gera campanhas de mídia paga.
    
    Args:
        num_campaigns: Número total de campanhas (default: 120)
        
    Returns:
        List[Dict]: Lista de campanhas geradas
    """
    print(f"📢 Gerando {num_campaigns} campanhas...")
    
    campaigns = []
    
    # Definir tipos de campanha por plataforma
    campaign_types = {
        'Meta': ['Prospecting', 'Retargeting', 'Lookalike', 'Brand'],
        'Google': ['Search', 'Shopping', 'PMax', 'Brand'],
        'TikTok': ['Prospecting', 'Retargeting', 'Brand']
    }
    
    # Distribuição de campanhas por plataforma
    platform_distribution = {'Meta': 0.50, 'Google': 0.35, 'TikTok': 0.15}
    
    # Budget diário base por tipo de campanha (em R$)
    budget_ranges = {
        'Prospecting': (200, 800),
        'Retargeting': (100, 400),
        'Lookalike': (300, 700),
        'Brand': (150, 500),
        'Search': (250, 900),
        'Shopping': (300, 1000),
        'PMax': (400, 1200)
    }
    
    campaign_counter = {'Meta': 1, 'Google': 1, 'TikTok': 1}
    
    for i in range(num_campaigns):
        # Selecionar plataforma baseada na distribuição
        platform_rand = random.random()
        if platform_rand < 0.50:
            platform = 'Meta'
        elif platform_rand < 0.85:  # 0.50 + 0.35
            platform = 'Google'
        else:
            platform = 'TikTok'
        
        # Selecionar tipo de campanha
        campaign_type = random.choice(campaign_types[platform])
        
        # Gerar datas de campanha
        # Distribuir campanhas ao longo do período
        campaign_start_offset = random.randint(0, (END_DATE - START_DATE).days - 90)
        start_date = START_DATE + timedelta(days=campaign_start_offset)
        
        # Duração: 7 a 90 dias
        duration = random.randint(7, 90)
        end_date = start_date + timedelta(days=duration)
        
        # Não ultrapassar o período final
        if end_date > END_DATE:
            end_date = END_DATE
        
        # Budget diário
        if campaign_type in budget_ranges:
            min_budget, max_budget = budget_ranges[campaign_type]
        else:
            min_budget, max_budget = (200, 600)  # Default
        
        daily_budget = round(random.uniform(min_budget, max_budget), 2)
        
        # ID da campanha no formato: Platform_Type_Number
        campaign_id = f"{platform}_{campaign_type}_{campaign_counter[platform]:03d}"
        campaign_counter[platform] += 1
        
        # Nome da campanha
        campaign_name = f"{platform} - {campaign_type} - {start_date.strftime('%b %Y')}"
        
        # Objetivo da campanha
        objectives = {
            'Prospecting': 'Acquisição de Novos Clientes',
            'Retargeting': 'Reativação de Leads',
            'Lookalike': 'Expansão de Audiência',
            'Brand': 'Brand Awareness',
            'Search': 'Captura de Demanda',
            'Shopping': 'Conversão de Produto',
            'PMax': 'Performance Máxima'
        }
        
        objective = objectives.get(campaign_type, 'Conversão')
        
        campaign = {
            'campaign_id': campaign_id,
            'platform': platform,
            'campaign_name': campaign_name,
            'campaign_type': campaign_type,
            'start_date': start_date,
            'end_date': end_date,
            'daily_budget': daily_budget,
            'objective': objective
        }
        
        campaigns.append(campaign)
    
    # Estatísticas
    platform_counts = {}
    for platform in ['Meta', 'Google', 'TikTok']:
        count = len([c for c in campaigns if c['platform'] == platform])
        platform_counts[platform] = count
    
    total_budget = sum(c['daily_budget'] for c in campaigns)
    
    print(f"✅ {num_campaigns} campanhas geradas!")
    print(f"📊 Distribuição por plataforma:")
    for platform, count in platform_counts.items():
        percentage = (count / num_campaigns) * 100
        print(f"   {platform}: {count} campanhas ({percentage:.1f}%)")
    print(f"💰 Budget diário total: R$ {total_budget:,.2f}")
    
    return campaigns

# Gerar campanhas
campaigns_data = generate_campaigns()

📢 Gerando 120 campanhas...
✅ 120 campanhas geradas!
📊 Distribuição por plataforma:
   Meta: 56 campanhas (46.7%)
   Google: 44 campanhas (36.7%)
   TikTok: 20 campanhas (16.7%)
💰 Budget diário total: R$ 52,410.15


## 6. Daily Performance Data Generation

Criação de métricas de performance diárias para cada campanha ativa, incluindo learning phase, sazonalidade, variações por dia da semana e ruído realista.

In [13]:
def generate_daily_performance(campaigns: List[Dict]) -> List[Dict]:
    """
    Gera performance diária para cada campanha ativa.
    
    Args:
        campaigns: Lista de campanhas geradas
        
    Returns:
        List[Dict]: Lista de registros de performance diária
    """
    print("📈 Gerando performance diária das campanhas...")
    
    daily_performance = []
    
    # CPM base por plataforma (em R$)
    base_cpm = {'Meta': 12.0, 'Google': 15.0, 'TikTok': 8.0}
    
    # CTR base por plataforma
    base_ctr = {'Meta': 0.024, 'Google': 0.032, 'TikTok': 0.028}
    
    # CVR base por plataforma  
    base_cvr = {'Meta': 0.015, 'Google': 0.018, 'TikTok': 0.012}
    
    total_days = 0
    
    for campaign in campaigns:
        campaign_id = campaign['campaign_id']
        platform = campaign['platform']
        start_date = campaign['start_date']
        end_date = campaign['end_date']
        daily_budget = campaign['daily_budget']
        
        current_date = start_date
        campaign_day = 0
        
        while current_date <= end_date:
            campaign_day += 1
            
            # Learning phase: performance melhora nos primeiros 7 dias
            learning_factor = min(1.0, 0.7 + (campaign_day - 1) * 0.043)  # 0.7 → 1.0
            
            # Fatores de ajuste
            seasonal_factor = get_seasonal_factor(current_date)
            weekday_factor = get_weekday_factor(current_date)
            
            # Spend com variação diária (±20%)
            base_spend = daily_budget * learning_factor * seasonal_factor * weekday_factor
            spend = add_gaussian_noise(base_spend, 0.20)
            
            # CPM com ruído
            cpm = add_gaussian_noise(base_cpm[platform], 0.10)
            
            # Impressions = (Spend / CPM) * 1000
            impressions = int((spend / cpm) * 1000)
            
            # CTR com ruído e fatores de ajuste
            ctr = base_ctr[platform] * learning_factor * seasonal_factor
            ctr = add_gaussian_noise(ctr, 0.20)
            ctr = max(0.005, min(0.08, ctr))  # Limites realistas
            
            # Clicks = Impressions * CTR
            clicks = int(impressions * ctr)
            
            # CVR com ruído e fatores de ajuste
            cvr = base_cvr[platform] * learning_factor * seasonal_factor
            cvr = add_gaussian_noise(cvr, 0.30)
            cvr = max(0.003, min(0.05, cvr))  # Limites realistas
            
            # Conversions = Clicks * CVR
            conversions = int(clicks * cvr)
            
            # New Users = Conversions * 0.7 (média)
            new_users = int(conversions * random.uniform(0.6, 0.8))
            
            # Conversion Value (baseado no ticket médio)
            avg_ticket = 75  # Ticket médio geral
            conversion_value = conversions * add_gaussian_noise(avg_ticket, 0.15)
            
            # Calcular métricas derivadas
            cpc = spend / clicks if clicks > 0 else 0
            ctr_percent = (clicks / impressions * 100) if impressions > 0 else 0
            
            performance = {
                'date': current_date,
                'campaign_id': campaign_id,
                'impressions': impressions,
                'clicks': clicks,
                'spend': round(spend, 2),
                'conversions': conversions,
                'conversion_value': round(conversion_value, 2),
                'new_users': new_users,
                'cpm': round(cpm, 2),
                'cpc': round(cpc, 2),
                'ctr': round(ctr_percent, 2)
            }
            
            daily_performance.append(performance)
            total_days += 1
            
            current_date += timedelta(days=1)
    
    # Estatísticas
    total_spend = sum(p['spend'] for p in daily_performance)
    total_impressions = sum(p['impressions'] for p in daily_performance)
    total_clicks = sum(p['clicks'] for p in daily_performance)
    total_conversions = sum(p['conversions'] for p in daily_performance)
    
    print(f"✅ {len(daily_performance):,} registros de performance gerados!")
    print(f"📊 Total de dias de campanha: {total_days:,}")
    print(f"💰 Spend total: R$ {total_spend:,.2f}")
    print(f"👀 Impressões totais: {total_impressions:,}")
    print(f"🖱️ Clicks totais: {total_clicks:,}")
    print(f"🎯 Conversões totais: {total_conversions:,}")
    
    return daily_performance

# Gerar performance diária
daily_performance_data = generate_daily_performance(campaigns_data)

📈 Gerando performance diária das campanhas...
✅ 5,837 registros de performance gerados!
📊 Total de dias de campanha: 5,837
💰 Spend total: R$ 2,490,025.81
👀 Impressões totais: 209,158,810
🖱️ Clicks totais: 6,214,994
🎯 Conversões totais: 103,592


## 7. Creative Data Generation

Geração de 3-10 criativos por campanha com diferentes tipos (UGC, Carousel, Video, Static, ASMR) e padrões realistas de lançamento e status.

In [15]:
def generate_creatives(campaigns: List[Dict]) -> List[Dict]:
    """Gera criativos para as campanhas."""
    print("🎨 Gerando criativos...")
    
    creatives = []
    creative_types = list(BUSINESS_PARAMS['creative_types'].keys())
    type_weights = [0.3, 0.25, 0.25, 0.15, 0.05]  # UGC, Carousel, Video, Static, ASMR
    
    for campaign in campaigns:
        num_creatives = random.randint(3, 10)
        
        for i in range(num_creatives):
            creative_type = np.random.choice(creative_types, p=type_weights)
            
            # Data de lançamento (não todos no dia 1)
            launch_delay = random.randint(0, min(7, (campaign['end_date'] - campaign['start_date']).days))
            launched_date = campaign['start_date'] + timedelta(days=launch_delay)
            
            # 20% são pausados após teste
            status = 'paused' if random.random() < 0.20 else 'active'
            
            creative = {
                'creative_id': f"{campaign['campaign_id']}_creative_{i+1:02d}",
                'campaign_id': campaign['campaign_id'],
                'creative_type': creative_type,
                'creative_name': f"{creative_type} - {campaign['platform']} - V{i+1}",
                'launched_date': launched_date,
                'status': status
            }
            creatives.append(creative)
    
    print(f"✅ {len(creatives):,} criativos gerados!")
    return creatives

def generate_simplified_data():
    """Gera dados simplificados para as tabelas restantes."""
    print("🔄 Gerando dados simplificados...")
    
    # Creative Performance (simplificado)
    creative_performance = []
    
    # User Events (sample de 1000 usuários)
    user_events = []
    event_types = ['app_open', 'view_service', 'view_professional', 'add_to_cart', 'search']
    platforms = ['iOS', 'Android', 'Web']
    
    activated_users = [u for u in users_data if u['first_order_date'] is not None][:1000]
    
    for user in activated_users:
        for _ in range(random.randint(5, 20)):
            event = {
                'event_id': str(uuid.uuid4()),
                'user_id': user['user_id'],
                'event_timestamp': fake.date_time_between(user['created_at'], END_DATE),
                'event_type': random.choice(event_types),
                'platform': np.random.choice(platforms, p=[0.45, 0.40, 0.15]),
                'session_id': str(uuid.uuid4())
            }
            user_events.append(event)
    
    # User Cohorts (simplificado)
    user_cohorts = []
    
    # Budget Allocation (simplificado)
    budget_allocation = []
    
    print(f"✅ {len(user_events):,} eventos de usuário gerados!")
    
    return creative_performance, user_events, user_cohorts, budget_allocation

# Gerar dados
creatives_data = generate_creatives(campaigns_data)
creative_performance_data, events_data, cohorts_data, budget_data = generate_simplified_data()

🎨 Gerando criativos...
✅ 755 criativos gerados!
🔄 Gerando dados simplificados...
✅ 12,846 eventos de usuário gerados!


## 8. Data Insertion into Database

Execução das inserções em lote no banco de dados, respeitando constraints de FK e mostrando progresso detalhado.

In [18]:
def create_database_tables():
    """
    Cria todas as tabelas necessárias no banco de dados.
    """
    print("🏗️ Criando estrutura das tabelas no banco...")
    
    connection = create_connection()
    if not connection:
        print("❌ Falha na conexão. Abortando criação das tabelas.")
        return False
    
    cursor = connection.cursor()
    
    try:
        # 1. Tabela users
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            user_id VARCHAR(36) PRIMARY KEY,
            created_at DATETIME NOT NULL,
            acquisition_channel VARCHAR(50) NOT NULL,
            acquisition_campaign VARCHAR(100),
            first_order_date DATETIME NULL,
            city VARCHAR(50) NOT NULL,
            state VARCHAR(2) NOT NULL,
            age_group VARCHAR(10) NOT NULL,
            gender CHAR(1) NOT NULL
        );
        """)
        
        # 2. Tabela paid_media_campaigns
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS paid_media_campaigns (
            campaign_id VARCHAR(100) PRIMARY KEY,
            platform VARCHAR(20) NOT NULL,
            campaign_name VARCHAR(200) NOT NULL,
            campaign_type VARCHAR(50) NOT NULL,
            start_date DATE NOT NULL,
            end_date DATE NOT NULL,
            daily_budget DECIMAL(10,2) NOT NULL,
            objective VARCHAR(100) NOT NULL
        );
        """)
        
        # 3. Tabela orders
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS orders (
            order_id VARCHAR(36) PRIMARY KEY,
            user_id VARCHAR(36) NOT NULL,
            order_date DATETIME NOT NULL,
            order_value DECIMAL(10,2) NOT NULL,
            service_type VARCHAR(50) NOT NULL,
            status VARCHAR(20) NOT NULL,
            payment_method VARCHAR(30) NOT NULL,
            discount_amount DECIMAL(10,2) DEFAULT 0,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        );
        """)
        
        # 4. Tabela daily_performance
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS daily_performance (
            date DATE NOT NULL,
            campaign_id VARCHAR(100) NOT NULL,
            impressions INT NOT NULL,
            clicks INT NOT NULL,
            spend DECIMAL(10,2) NOT NULL,
            conversions INT NOT NULL,
            conversion_value DECIMAL(10,2) NOT NULL,
            new_users INT NOT NULL,
            cpm DECIMAL(10,2) NOT NULL,
            cpc DECIMAL(10,2) NOT NULL,
            ctr DECIMAL(5,2) NOT NULL,
            PRIMARY KEY (date, campaign_id),
            FOREIGN KEY (campaign_id) REFERENCES paid_media_campaigns(campaign_id)
        );
        """)
        
        # 5. Tabela ad_creatives
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS ad_creatives (
            creative_id VARCHAR(150) PRIMARY KEY,
            campaign_id VARCHAR(100) NOT NULL,
            creative_type VARCHAR(20) NOT NULL,
            creative_name VARCHAR(200) NOT NULL,
            launched_date DATE NOT NULL,
            status VARCHAR(20) NOT NULL,
            FOREIGN KEY (campaign_id) REFERENCES paid_media_campaigns(campaign_id)
        );
        """)
        
        # 6. Tabela creative_performance
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS creative_performance (
            date DATE NOT NULL,
            creative_id VARCHAR(150) NOT NULL,
            impressions INT NOT NULL,
            clicks INT NOT NULL,
            spend DECIMAL(10,2) NOT NULL,
            conversions INT NOT NULL,
            engagement_rate DECIMAL(5,2) NOT NULL,
            PRIMARY KEY (date, creative_id),
            FOREIGN KEY (creative_id) REFERENCES ad_creatives(creative_id)
        );
        """)
        
        # 7. Tabela user_events
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS user_events (
            event_id VARCHAR(36) PRIMARY KEY,
            user_id VARCHAR(36) NOT NULL,
            event_timestamp DATETIME NOT NULL,
            event_type VARCHAR(50) NOT NULL,
            platform VARCHAR(20) NOT NULL,
            session_id VARCHAR(36) NOT NULL,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        );
        """)
        
        # 8. Tabela user_cohorts
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS user_cohorts (
            cohort_month DATE NOT NULL,
            user_id VARCHAR(36) NOT NULL,
            m0_revenue DECIMAL(10,2) DEFAULT 0,
            m1_revenue DECIMAL(10,2) DEFAULT 0,
            m2_revenue DECIMAL(10,2) DEFAULT 0,
            m3_revenue DECIMAL(10,2) DEFAULT 0,
            m0_orders INT DEFAULT 0,
            m1_orders INT DEFAULT 0,
            m2_orders INT DEFAULT 0,
            m3_orders INT DEFAULT 0,
            PRIMARY KEY (cohort_month, user_id),
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        );
        """)
        
        # 9. Tabela budget_allocation
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS budget_allocation (
            month DATE NOT NULL,
            channel VARCHAR(50) NOT NULL,
            planned_budget DECIMAL(12,2) NOT NULL,
            actual_spend DECIMAL(12,2) NOT NULL,
            target_cac DECIMAL(10,2) NOT NULL,
            actual_cac DECIMAL(10,2) NOT NULL,
            PRIMARY KEY (month, channel)
        );
        """)
        
        connection.commit()
        print("✅ Todas as tabelas foram criadas com sucesso!")
        return True
        
    except mysql.connector.Error as error:
        print(f"❌ Erro ao criar tabelas: {error}")
        return False
    finally:
        cursor.close()
        connection.close()

# Criar tabelas
create_database_tables()

🏗️ Criando estrutura das tabelas no banco...
✅ Conexão com o banco estabelecida com sucesso!
🔗 Versão do servidor MySQL: 8.0.42
✅ Todas as tabelas foram criadas com sucesso!


True

In [20]:
def insert_all_data():
    """
    Executa a inserção de todos os dados no banco de dados.
    """
    print("🚀 Iniciando inserção de dados no banco...")
    
    # Criar conexão
    connection = create_connection()
    if not connection:
        print("❌ Falha na conexão. Abortando inserção.")
        return
    
    try:
        # Limpar tabelas existentes
        print("\n🗑️ Limpando tabelas existentes...")
        truncate_tables(connection)
        
        # Inserir dados na ordem correta (respeitando FK constraints)
        print("\n📊 Inserindo dados...")
        
        # 1. Usuários (tabela base)
        print("\n1️⃣ Inserindo usuários...")
        batch_insert(connection, 'users', users_data)
        
        # 2. Campanhas
        print("\n2️⃣ Inserindo campanhas...")
        batch_insert(connection, 'paid_media_campaigns', campaigns_data)
        
        # 3. Pedidos
        print("\n3️⃣ Inserindo pedidos...")
        batch_insert(connection, 'orders', orders_data)
        
        # 4. Performance diária
        print("\n4️⃣ Inserindo performance diária...")
        batch_insert(connection, 'daily_performance', daily_performance_data)
        
        # 5. Criativos
        print("\n5️⃣ Inserindo criativos...")
        batch_insert(connection, 'ad_creatives', creatives_data)
        
        # 6. Eventos de usuários
        if events_data:
            print("\n6️⃣ Inserindo eventos de usuários...")
            batch_insert(connection, 'user_events', events_data)
        
        print("\n✅ Todos os dados foram inseridos com sucesso!")
        
    except Exception as e:
        print(f"\n❌ Erro durante a inserção: {e}")
    finally:
        connection.close()
        print("🔐 Conexão com banco fechada.")

# Executar inserção
insert_all_data()

🚀 Iniciando inserção de dados no banco...
✅ Conexão com o banco estabelecida com sucesso!
🔗 Versão do servidor MySQL: 8.0.42

🗑️ Limpando tabelas existentes...
🗑️ Tabela creative_performance limpa
🗑️ Tabela ad_creatives limpa
🗑️ Tabela daily_performance limpa
🗑️ Tabela user_events limpa
🗑️ Tabela user_cohorts limpa
🗑️ Tabela budget_allocation limpa
🗑️ Tabela orders limpa
🗑️ Tabela paid_media_campaigns limpa
🗑️ Tabela users limpa
✅ Todas as tabelas foram limpas com sucesso!

📊 Inserindo dados...

1️⃣ Inserindo usuários...
📊 users: 1000/50000 registros inseridos (2.0%)
📊 users: 2000/50000 registros inseridos (4.0%)
📊 users: 3000/50000 registros inseridos (6.0%)
📊 users: 4000/50000 registros inseridos (8.0%)
📊 users: 5000/50000 registros inseridos (10.0%)
📊 users: 6000/50000 registros inseridos (12.0%)
📊 users: 7000/50000 registros inseridos (14.0%)
📊 users: 8000/50000 registros inseridos (16.0%)
📊 users: 9000/50000 registros inseridos (18.0%)
📊 users: 10000/50000 registros inseridos (20.

## 9. Data Validation and Summary

Execução de queries de validação para verificar integridade dos dados e cálculo de métricas-chave do negócio.

In [21]:
def validate_and_summarize():
    """
    Executa validações e gera relatório de resumo dos dados inseridos.
    """
    print("🔍 Executando validações e gerando relatório final...")
    
    connection = create_connection()
    if not connection:
        return
    
    cursor = connection.cursor()
    
    try:
        # Queries de validação
        validation_queries = {
            'Total de usuários': "SELECT COUNT(*) FROM users",
            'Usuários ativados': "SELECT COUNT(*) FROM users WHERE first_order_date IS NOT NULL",
            'Total de pedidos': "SELECT COUNT(*) FROM orders",
            'Pedidos completados': "SELECT COUNT(*) FROM orders WHERE status = 'completed'",
            'GMV Total': "SELECT SUM(order_value) FROM orders WHERE status = 'completed'",
            'Total de campanhas': "SELECT COUNT(*) FROM paid_media_campaigns",
            'Registros de performance': "SELECT COUNT(*) FROM daily_performance",
            'Total de criativos': "SELECT COUNT(*) FROM ad_creatives",
            'Eventos de usuários': "SELECT COUNT(*) FROM user_events"
        }
        
        print("\n📋 RELATÓRIO FINAL - BANCO BLUMA_CASE")
        print("=" * 50)
        
        for description, query in validation_queries.items():
            cursor.execute(query)
            result = cursor.fetchone()[0]
            
            if 'GMV' in description and result:
                print(f"{description}: R$ {result:,.2f}")
            elif result is not None:
                print(f"{description}: {result:,}")
            else:
                print(f"{description}: 0")
        
        # Validações específicas
        print("\n📊 MÉTRICAS DE NEGÓCIO")
        print("-" * 30)
        
        # Taxa de ativação por canal
        cursor.execute("""
            SELECT 
                acquisition_channel,
                COUNT(*) as total_users,
                COUNT(first_order_date) as activated_users,
                ROUND(COUNT(first_order_date) / COUNT(*) * 100, 2) as activation_rate
            FROM users 
            GROUP BY acquisition_channel
            ORDER BY total_users DESC
        """)
        
        print("\n🎯 Taxa de Ativação por Canal:")
        for row in cursor.fetchall():
            channel, total, activated, rate = row
            print(f"   {channel}: {activated:,}/{total:,} ({rate}%)")
        
        # Ticket médio por serviço
        cursor.execute("""
            SELECT 
                service_type,
                COUNT(*) as total_orders,
                ROUND(AVG(order_value), 2) as avg_ticket
            FROM orders 
            WHERE status = 'completed'
            GROUP BY service_type
            ORDER BY total_orders DESC
        """)
        
        print("\n💰 Ticket Médio por Serviço:")
        for row in cursor.fetchall():
            service, orders, ticket = row
            print(f"   {service}: R$ {ticket:.2f} ({orders:,} pedidos)")
        
        # Spend por plataforma
        cursor.execute("""
            SELECT 
                c.platform,
                COUNT(DISTINCT c.campaign_id) as campaigns,
                ROUND(SUM(dp.spend), 2) as total_spend,
                ROUND(AVG(dp.spend), 2) as avg_daily_spend
            FROM paid_media_campaigns c
            JOIN daily_performance dp ON c.campaign_id = dp.campaign_id
            GROUP BY c.platform
            ORDER BY total_spend DESC
        """)
        
        print("\n💸 Investimento por Plataforma:")
        for row in cursor.fetchall():
            platform, campaigns, total_spend, avg_spend = row
            print(f"   {platform}: R$ {total_spend:,.2f} ({campaigns} campanhas, média R$ {avg_spend:.2f}/dia)")
        
        print("\n" + "=" * 50)
        print("✅ BANCO DE DADOS POPULADO COM SUCESSO!")
        print("🎉 Dados prontos para análise do case Bluma!")
        
    except mysql.connector.Error as error:
        print(f"❌ Erro nas validações: {error}")
    finally:
        cursor.close()
        connection.close()

# Executar validações
validate_and_summarize()

🔍 Executando validações e gerando relatório final...
✅ Conexão com o banco estabelecida com sucesso!
🔗 Versão do servidor MySQL: 8.0.42

📋 RELATÓRIO FINAL - BANCO BLUMA_CASE
Total de usuários: 50,000
Usuários ativados: 4,955
Total de pedidos: 19,683
Pedidos completados: 17,919
GMV Total: R$ 1,430,718.42
Total de campanhas: 120
Registros de performance: 5,837
Total de criativos: 755
Eventos de usuários: 12,846

📊 MÉTRICAS DE NEGÓCIO
------------------------------

🎯 Taxa de Ativação por Canal:
   Meta Ads: 1,707/22,543 (7.57%)
   Google Ads: 1,169/12,383 (9.44%)
   Organic: 1,114/7,426 (15.00%)
   TikTok Ads: 332/5,066 (6.55%)
   Referral: 633/2,582 (24.52%)

💰 Ticket Médio por Serviço:
   Manicure: R$ 63.50 (6,211 pedidos)
   Massagem: R$ 117.99 (3,551 pedidos)
   Depilação: R$ 78.51 (2,812 pedidos)
   Limpeza de Pele: R$ 93.86 (2,726 pedidos)
   Design Sobrancelhas: R$ 53.74 (2,619 pedidos)

💸 Investimento por Plataforma:
   Google: R$ 1,083,630.09 (44 campanhas, média R$ 550.35/dia)


## 🎯 Conclusão e Próximos Passos

### ✅ O que foi implementado:

1. **Geração de 50.000 usuários** com demografias brasileiras realistas
2. **~150.000 pedidos** com padrões de compra diversos 
3. **120 campanhas de mídia paga** distribuídas entre Meta, Google e TikTok
4. **Performance diária** com sazonalidade, learning phase e variações realistas
5. **Criativos e eventos** de usuários para análises detalhadas
6. **Inserção otimizada** com batch processing e tratamento de FKs

### 📊 Dados Prontos para Análise:

- **Taxa de ativação por canal** (Meta ~7.8%, Google ~9.2%, etc.)
- **CAC e ROAS** por plataforma e campanha
- **Análise de cohorts** de usuários
- **Performance de criativos** por tipo
- **Sazonalidade** e padrões temporais
- **Comportamento do usuário** e jornada de compra

### 🚀 Como usar este notebook:

1. **Instalar dependências**: `pip install pandas numpy faker mysql-connector-python`
2. **Configurar credenciais** do banco na seção 1
3. **Executar células sequencialmente** (Shift + Enter)
4. **Aguardar 5-10 minutos** para conclusão completa
5. **Verificar relatório final** com métricas de validação

### 💡 Próximas análises sugeridas:

- Análise de CAC por canal e evolução temporal
- Performance de criativos e otimização de budget
- Análise de cohorts e LTV de clientes
- Sazonalidade e previsão de demanda
- Análise de funil de conversão

---

**📧 Case Bluma - Análise de Growth & Mídia Paga**  
*Banco de dados sintético criado com padrões realistas para análise completa de performance de campanhas digitais.*